In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import torchvision
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
from timm.models import create_model
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from tqdm import tqdm
import json
from PIL import Image

### Modelo

In [2]:
class ViTMultiClassClassifier(nn.Module):
    """
    Clasificador Multi-Clase Vision Transformer con Detección de Anomalías
    
    Un modelo de red neuronal PyTorch que combina clasificación multi-clase con detección de anomalías
    usando un backbone Vision Transformer (ViT). Esta implementación sigue un enfoque híbrido para
    detección de anomalías combinando probabilidades de clasificación con similitud coseno a features
    de clase normal.
    
    El modelo está diseñado para clasificación de defectos en cuero con 6 clases:
    - folding_marks (clase 0)
    - grain_off (clase 1) 
    - growth_marks (clase 2)
    - loose_grain (clase 3)
    - pinhole (clase 4)
    - non_defective (clase 5)
    
    Arquitectura:
    - Backbone: ViT-Base/16 (features de 768 dimensiones)
    - Clasificador: Cabeza MLP personalizada con dropout y activación ReLU
    - Detección de Anomalías: Método híbrido usando clasificación + similitud coseno
    
    Métodos:
    - forward(): Pase hacia adelante completo devolviendo logits y features
    - extract_features(): Solo extracción de features (sin clasificación)
    - store_normal_features(): Almacenar features de clase normal para detección de anomalías
    - classify_multiclass(): Clasificación multi-clase estándar
    - detect_anomaly_hybrid(): Detección de anomalías híbrida combinando múltiples métodos
    
    Parámetros:
        num_classes (int): Número de clases de salida (por defecto: 6)
        pretrained (bool): Si usar pesos pre-entrenados de ViT (por defecto: True)
    
    Atributos:
        backbone: Extractor de features ViT
        classifier: Cabeza de clasificación personalizada
        normal_features: Features almacenadas de clase normal para detección de anomalías
        class_names: Lista de nombres de clases para referencia
    
    Nota:
        El modelo espera que 'non_defective' sea clase 4 basado en la estructura del dataset de Kaggle.
        Para detección de anomalías, el modelo almacena features de muestras normales y calcula
        puntuaciones de anomalía usando una combinación ponderada de confianza de clasificación y
        similitud coseno con features normales.
    """
    def __init__(self, num_classes=6, pretrained=True):
        super(ViTMultiClassClassifier, self).__init__()

        # ViT-Base/16 como feature extractor
        self.backbone = create_model(
            'vit_base_patch16_224',
            pretrained=pretrained,
            num_classes=0  # Sin head de clasificación
        )

        # Head de clasificación personalizado para 6 clases
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

        # Para almacenar features de clases normales (detección de anomalías)
        self.normal_features = None
        self.class_names = [
            "folding_marks",
            "grain_off",
            "growth_marks",
            "loose_grain",
            "pinhole",
            "non_defective",
        ]

    def forward(self, x):
        """Forward pass completo: features + clasificación"""
        features = self.backbone(x)  # [batch_size, 768]
        logits = self.classifier(features)  # [batch_size, 6]
        return logits, features

    def extract_features(self, x):
        """Solo extracción de features sin clasificación"""
        return self.backbone(x)

    def store_normal_features(self, dataloader, device):
        """
        Extraer y almacenar representaciones de características de imágenes normales (sin defectos).
        Este método procesa un dataloader para extraer características de imágenes etiquetadas como
        'non_defective' (clase 4) y las almacena para uso posterior en detección de anomalías.
        El modelo se establece en modo de evaluación durante la extracción de características.
        
        Args:
            dataloader (torch.utils.data.DataLoader): DataLoader que contiene imágenes y etiquetas
            device (torch.device): Dispositivo para ejecutar el cálculo (CPU o GPU)
            
        Returns:
            None: Almacena las características extraídas en el atributo self.normal_features
            
        Efectos secundarios:
            - Establece el modelo en modo de evaluación
            - Llena self.normal_features con tensores de características concatenados
            - Imprime el progreso de extracción y estadísticas
            - Crea características dummy si no se encuentran imágenes normales
            
        Nota:
            - Solo procesa imágenes con etiqueta == 4 (clase non_defective en el dataset de Kaggle)
            - Las características se mueven a CPU para almacenamiento
            - Imprime estadísticas de características incluyendo desviación estándar y norma
        """
        self.eval()
        normal_features = []
        normal_count = 0

        print("Extrayendo features de imágenes normales (clase 'non_defective')...")
        with torch.no_grad():
            for images, labels in tqdm(dataloader, desc="Procesando features normales"):
                images = images.to(device)

                # Procesar todas las imágenes y filtrar por clase después
                features = self.extract_features(images)

                # Solo almacenar features de clase 'non_defective' (clase 4 en dataset Kaggle)
                for i, label in enumerate(labels):
                    if label.item() == 4:  # non_defective en dataset Kaggle
                        normal_features.append(features[i:i+1].cpu())
                        normal_count += 1

        if normal_features:
            self.normal_features = torch.cat(normal_features, dim=0)
            feature_std = torch.std(self.normal_features, dim=0).mean()
            feature_mean = torch.mean(self.normal_features, dim=0).norm()

            print(f"✓ Almacenadas {len(self.normal_features)} features normales")
            print(f"  - Desviación estándar promedio: {feature_std:.4f}")
            print(f"  - Norma promedio: {feature_mean:.4f}")
        else:
            print(" No se encontraron imágenes de clase 'non_defective'")
            # Crear features dummy para evitar errores
            self.normal_features = torch.randn(10, 768)

    def classify_multiclass(self, x):
        """Clasificación multi-clase estándar"""
        logits, features = self.forward(x)
        probs = F.softmax(logits, dim=1)
        predicted_classes = torch.argmax(probs, dim=1)

        return {
            'logits': logits,
            'probabilities': probs,
            'predicted_classes': predicted_classes,
            'features': features
        }

    def detect_anomaly_hybrid(self, x):
        """
        Detectar anomalías usando un enfoque híbrido que combina métodos de clasificación y similitud.
        Este método implementa una estrategia de detección de anomalías multifacética que combina:
        1. Puntuación basada en clasificación usando probabilidad de clase normal
        2. Puntuación de similitud coseno contra características normales conocidas
        3. Combinación híbrida ponderada de ambos enfoques
        Args:
            x (torch.Tensor): Tensor de entrada de forma (batch_size, channels, height, width)
                     que contiene las imágenes a analizar para detectar anomalías.
        Returns:
            dict: Un diccionario que contiene las siguientes claves:
            - 'anomaly_scores' (torch.Tensor): Puntuaciones de anomalía híbridas (0-1, mayor = más anómalo)
            - 'similarity_scores' (torch.Tensor): Puntuaciones de anomalía basadas en similitud coseno
            - 'classification_scores' (torch.Tensor): Puntuaciones de anomalía basadas en clasificación
            - 'predicted_classes' (torch.Tensor): Índices de clases predichas
            - 'class_probabilities' (torch.Tensor): Probabilidades softmax para todas las clases
            - 'features' (torch.Tensor): Representaciones de características extraídas
            - 'normal_class_prob' (torch.Tensor): Probabilidad de clase normal (clase 4)
        Notas:
            - Usa peso alpha=0.7 para puntuación de similitud y beta=0.3 para puntuación de clasificación
            - Asume que la clase 4 representa la clase normal 'non_defective'
            - Si normal_features es None, recurre a puntuación solo de clasificación
            - Todas las puntuaciones de anomalía están normalizadas al rango [0, 1] donde 1 indica alta probabilidad de anomalía
        """
        logits, features = self.forward(x)
        probs = F.softmax(logits, dim=1)
        predicted_classes = torch.argmax(probs, dim=1)

        # Método 1: Probabilidad de clase normal
        normal_class_prob = probs[:, 4]  # Probabilidad de 'non_defective' (clase 4 en Kaggle)
        classification_anomaly_score = 1.0 - normal_class_prob

        # Método 2: Similitud coseno con features normales
        if self.normal_features is not None:
            features_norm = F.normalize(features, p=2, dim=1)
            normal_features_norm = F.normalize(self.normal_features.to(features.device), p=2, dim=1)

            similarities = torch.mm(features_norm, normal_features_norm.T)
            max_similarities, _ = torch.max(similarities, dim=1)
            similarity_anomaly_score = 1.0 - max_similarities
        else:
            similarity_anomaly_score = classification_anomaly_score

        # Método 3: Combinación híbrida (como sugiere el paper)
        # Combinar clasificación y similitud con pesos
        alpha = 0.7  # Peso para similitud coseno
        beta = 0.3   # Peso para clasificación

        hybrid_anomaly_score = (alpha * similarity_anomaly_score + 
                               beta * classification_anomaly_score)

        return {
            'anomaly_scores': hybrid_anomaly_score,
            'similarity_scores': similarity_anomaly_score,
            'classification_scores': classification_anomaly_score,
            'predicted_classes': predicted_classes,
            'class_probabilities': probs,
            'features': features,
            'normal_class_prob': normal_class_prob
        }

### Funciones creadoras de datasets

In [3]:
class LeatherDefectDataset(Dataset):
    """
    Una clase Dataset de PyTorch para cargar y preprocesar imágenes de defectos en cuero.
    Este dataset carga imágenes de un dataset de defectos en cuero de Kaggle con 6 clases:
    folding_marks, grain_off, growth_marks, loose_grains, non_defective, y pinhole.
    Automáticamente divide los datos en conjuntos de entrenamiento y validación manteniendo
    la distribución de clases.
    
    Args:
        root_path (str): Ruta al directorio raíz que contiene las carpetas de clases
        is_train (bool, opcional): Si es True, carga el conjunto de entrenamiento; si es False, 
            carga el conjunto de validación. Por defecto True.
        validation_split (float, opcional): Fracción de datos a usar para validación (0-1). 
            Por defecto 0.2.
        transform (callable, opcional): Transformación opcional a aplicar a las imágenes. 
            Por defecto None.
        random_seed (int, opcional): Semilla aleatoria para divisiones reproducibles de 
            entrenamiento/validación. Por defecto 42.
    
    Atributos:
        folder_to_class (dict): Mapeo de nombres de carpetas a índices de clases
        class_names (list): Lista de nombres de clases en orden de índices de clases
        image_paths (list): Lista de rutas a todas las imágenes en la división actual
        labels (list): Lista de etiquetas de clase correspondientes para cada imagen
    
    Métodos:
        _load_data(): Método interno para cargar y dividir el dataset
        __len__(): Devuelve el número total de muestras en la división actual
        __getitem__(idx): Devuelve una tupla de (imagen, etiqueta) para el índice dado
    
    Nota:
        El dataset espera la siguiente estructura de carpetas:
        root_path/
        ├── folding_marks/
        ├── grain_off/
        ├── growth_marks/
        ├── loose_grains/
        ├── non_defective/
        └── pinhole/
    """
    def __init__(self, root_path, is_train=True, validation_split=0.2, transform=None, random_seed=42):
        self.root_path = root_path
        self.is_train = is_train
        self.validation_split = validation_split
        self.transform = transform
        self.random_seed = random_seed
        
        # Mapeo exacto de las carpetas del dataset de Kaggle a clases
        self.folder_to_class = {
            'folding_marks': 0,      # folding_marks
            'grain_off': 1,          # grain_off  
            'growth_marks': 2,       # growth_marks
            'loose_grains': 3,       # loose_grain (nota: 'grains' en plural en Kaggle)
            'non_defective': 4,      # non_defective
            'pinhole': 5             # pinhole
        }
        
        # Nombres de clases para el modelo (mantenemos consistencia con el paper)
        self.class_names = [
            'folding_marks',    # 0
            'grain_off',        # 1  
            'growth_marks',     # 2
            'loose_grain',      # 3 (singular como en el paper)
            'non_defective',    # 4
            'pinhole'           # 5
        ]
        
        self._load_data()
    
    def _load_data(self):
        """
        Cargar y dividir el dataset de imágenes en conjuntos de entrenamiento y validación.
        Este método carga imágenes desde el directorio raíz especificado, organizándolas por 
        carpetas de clases y dividiéndolas en conjuntos de entrenamiento y validación basándose 
        en la proporción de división de validación configurada. La división se realiza de manera 
        consistente usando una semilla aleatoria.
        
        El método puebla los siguientes atributos de instancia:
        - self.image_paths: Lista de rutas de archivo a las imágenes seleccionadas
        - self.labels: Lista de IDs de clase correspondientes para cada imagen
        
        Estructura de Directorio Esperada:
            root_path/
            ├── carpeta_clase_1/
            │   ├── imagen1.jpg
            │   └── imagen2.png
            └── carpeta_clase_2/
                ├── imagen3.jpeg
                └── imagen4.jpg
        
        Proceso:
        1. Escanea cada carpeta de clase definida en self.folder_to_class
        2. Recopila todos los archivos de imagen válidos (.png, .jpg, .jpeg)
        3. Mezcla aleatoriamente las imágenes dentro de cada clase usando self.random_seed
        4. Divide cada clase según la proporción self.validation_split
        5. Selecciona el subconjunto de entrenamiento o validación basado en la bandera self.is_train
        6. Imprime estadísticas detalladas sobre el proceso de carga y división
        
        Excepciones:
            Maneja implícitamente directorios faltantes imprimiendo advertencias y continuando
            con listas de imágenes vacías para esas clases.
        
        Nota:
            La división de validación se aplica por clase para mantener la distribución de clases
            tanto en los conjuntos de entrenamiento como de validación.
        """
        self.image_paths = []
        self.labels = []
        
        print(f"Cargando desde: {self.root_path}")
        print(f"Carpetas esperadas: {list(self.folder_to_class.keys())}")
        
        # Recopilar todas las imágenes por clase
        all_images_by_class = {}
        
        for folder_name, class_id in self.folder_to_class.items():
            class_dir = os.path.join(self.root_path, folder_name)
            if os.path.exists(class_dir):
                images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                all_images_by_class[class_id] = images
                print(f"   {folder_name}: {len(images)} imágenes → clase {class_id} ({self.class_names[class_id]})")
            else:
                print(f"   No encontrado: {class_dir}")
                all_images_by_class[class_id] = []
        
        # Dividir cada clase en train/validation
        np.random.seed(self.random_seed)
        
        for class_id, images in all_images_by_class.items():
            if len(images) > 0:
                # Mezclar imágenes
                images = np.array(images)
                indices = np.random.permutation(len(images))
                images = images[indices]
                
                # Dividir en train/validation
                n_val = int(len(images) * self.validation_split)
                
                if self.is_train:
                    # Usar para entrenamiento (80%)
                    selected_images = images[n_val:]
                else:
                    # Usar para validación (20%)
                    selected_images = images[:n_val]
                
                self.image_paths.extend(selected_images.tolist())
                self.labels.extend([class_id] * len(selected_images))
        
        print(f"\n DIVISIÓN TRAIN/VALIDATION:")
        print(f"Modo: {'Entrenamiento' if self.is_train else 'Validación'}")
        print(f"Total imágenes: {len(self.image_paths)}")
        
        # Mostrar distribución por clase
        unique_labels, counts = np.unique(self.labels, return_counts=True)
        for class_id, count in zip(unique_labels, counts):
            print(f"  {self.class_names[class_id]}: {count} imágenes")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        """
        Recuperar una imagen y su etiqueta correspondiente en el índice especificado.
        Args:
            idx (int): Índice del elemento a recuperar del dataset.
        Returns:
            tuple: Una tupla que contiene:
                - image (torch.Tensor o PIL.Image): Los datos de imagen procesados. Si se aplica
                  transform, devuelve un tensor; de lo contrario devuelve una imagen PIL en formato RGB.
                - label: La etiqueta correspondiente para la imagen en el índice dado.
        Nota:
            - Las imágenes se convierten automáticamente a formato RGB al cargarlas.
            - Si se especifica una transformación durante la inicialización del dataset, se
              aplicará a la imagen antes de devolverla.
        """
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [4]:
class MVTecTestDataset(Dataset):
    """
    Clase Dataset para cargar datos de prueba de MVTec Anomaly Detection.
    Este dataset está específicamente diseñado para tareas de clasificación binaria (normal vs anomalía)
    usando la división de prueba del dataset MVTec AD. Carga imágenes de una categoría especificada
    y asigna etiquetas binarias donde las muestras 'good' son etiquetadas como normales (0) y todos
    los tipos de defectos son etiquetados como anomalías (1).
    Args:
        root_path (str): Ruta del directorio raíz que contiene el dataset MVTec AD
        category (str, opcional): Categoría de producto a cargar (ej. 'leather', 'bottle'). 
                                 Por defecto 'leather'
        transform (callable, opcional): Transformación opcional a aplicar a las imágenes.
                                       Por defecto None
    Atributos:
        root_path (str): Ruta del directorio raíz del dataset
        category (str): Categoría de producto siendo cargada
        transform (callable): Pipeline de transformación de imágenes
        class_names (list): Nombres de clases binarias ['normal', 'anomaly']
        image_paths (list): Lista de rutas a todas las imágenes cargadas
        labels (list): Lista de etiquetas binarias (0=normal, 1=anomalía)
        defect_types (list): Lista de nombres originales de tipos de defecto para seguimiento
    Retorna:
        tuple: (imagen, etiqueta, tipo_defecto) donde:
            - imagen: Imagen PIL o tensor transformado
            - etiqueta: Etiqueta binaria (0 para normal, 1 para anomalía)
            - tipo_defecto: Cadena del tipo de defecto original del dataset MVTec
    Ejemplo:
        >>> dataset = MVTecTestDataset(
        ...     root_path='/ruta/a/mvtec',
        ...     category='leather',
        ...     transform=transforms.ToTensor()
        ... )
        >>> imagen, etiqueta, tipo_defecto = dataset[0]
    """
    def __init__(self, root_path, category='leather', transform=None):
        self.root_path = root_path
        self.category = category
        self.transform = transform
        
        # Solo clases binarias para MVTec: normal vs anomalía
        self.class_names = ['normal', 'anomaly']
        
        self._load_data()
    
    def _load_data(self):
        """
        Cargar imágenes y etiquetas del dataset MVTec desde el directorio de prueba.
        Este método recorre la estructura del directorio de prueba y carga las rutas de imágenes,
        etiquetas y tipos de defectos para la categoría especificada. Las imágenes en el
        subdirectorio 'good' se etiquetan como normales (0), mientras que todos los demás
        subdirectorios se etiquetan como anomalías (1).
        
        El método puebla los siguientes atributos de instancia:
        - image_paths: Lista de rutas completas a todos los archivos de imagen
        - labels: Lista de etiquetas binarias (0 para normal, 1 para anomalía)
        - defect_types: Lista de nombres de tipos de defectos para seguimiento
        
        Estructura de directorio esperada:
        root_path/category/test/
        ├── good/           # Imágenes normales (etiqueta = 0)
        ├── tipo_defecto1/  # Imágenes anómalas (etiqueta = 1)
        ├── tipo_defecto2/  # Imágenes anómalas (etiqueta = 1)
        └── ...
        
        Solo se procesan imágenes PNG. Se imprime información de progreso en consola
        mostrando el número de imágenes cargadas para cada tipo de defecto.
        """
        self.image_paths = []
        self.labels = []
        self.defect_types = []  # Para tracking de tipos de defecto
        
        test_dir = os.path.join(self.root_path, self.category, 'test')
        print(f"Cargando MVTec test desde: {test_dir}")
        
        for defect_type in os.listdir(test_dir):
            defect_path = os.path.join(test_dir, defect_type)
            if os.path.isdir(defect_path):
                images = [os.path.join(defect_path, f) for f in os.listdir(defect_path) 
                         if f.endswith('.png')]
                
                # MVTec: 'good' = normal (0), todo lo demás = anomalía (1)
                label = 0 if defect_type == 'good' else 1
                
                self.image_paths.extend(images)
                self.labels.extend([label] * len(images))
                self.defect_types.extend([defect_type] * len(images))
                print(f"  {defect_type}: {len(images)} imágenes → clase {label}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        """
        Recuperar un elemento individual del dataset por índice.
        
        Args:
            idx (int): Índice del elemento a recuperar del dataset.
            
        Returns:
            tuple: Una tupla que contiene:
            - image (torch.Tensor o PIL.Image): Los datos de imagen procesados
            - label (any): La etiqueta asociada con la imagen  
            - defect_type (any): El tipo de defecto para la imagen
            
        Nota:
            La imagen se carga desde la ruta del archivo, se convierte a formato RGB,
            y opcionalmente se transforma si se proporciona una función de transformación.
        """
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label, self.defect_types[idx]  # Incluir tipo de defecto

### Función de entrenamiento

In [5]:
def train_model(model, train_loader, val_loader, lr, wd, num_epochs, device, output_dir, logs_dir, model_name):
    """
    Entrenar un modelo de PyTorch con registro integral y validación.
    Esta función realiza bucles de entrenamiento y validación durante un número dado de épocas,
    rastrea métricas usando TensorBoard, y guarda el modelo con mejor rendimiento basado en
    la precisión de validación.
    
    Args:
        model: Modelo de PyTorch a ser entrenado
        train_loader: DataLoader para datos de entrenamiento
        val_loader: DataLoader para datos de validación
        lr (float): Tasa de aprendizaje para el optimizador
        wd (float): Decaimiento de peso para el optimizador
        num_epochs (int): Número de épocas de entrenamiento
        device: Dispositivo de PyTorch (CPU o CUDA) para entrenamiento
        output_dir (str): Directorio para guardar el mejor checkpoint del modelo
        logs_dir (str): Directorio para guardar logs de TensorBoard
        model_name (str): Prefijo del nombre para modelo guardado y logs
    
    Returns:
        model: El modelo de PyTorch entrenado
    
    Notas:
        - Usa optimizador AdamW con programador de tasa de aprendizaje Cosine Annealing
        - Implementa CrossEntropyLoss para clasificación
        - Guarda el modelo con mayor precisión de validación
        - Registra pérdida de entrenamiento/validación, precisión y tasa de aprendizaje en TensorBoard
        - Incluye registro de histogramas y visualización de imágenes de muestra
        - Crea directorio de salida si no existe
    """
    writer = SummaryWriter(log_dir=f'{logs_dir}/{model_name}') # Para tensorboard
    os.makedirs(output_dir, exist_ok=True)

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        # =============
        # Entrenamiento
        # =============
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Train"):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            logits, _ = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # ==========
        # Validación
        # ==========
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1} - Val"):
                images, labels = images.to(device), labels.to(device)
                
                logits, _ = model(images)
                loss = criterion(logits, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(logits, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # =====================================
        # Calcular métricas y registrar logging
        # =====================================
        train_acc = 100 * train_correct / train_total
        val_acc = 100 * val_correct / val_total
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train - Loss: {train_loss/len(train_loader):.4f}, Acc: {train_acc:.2f}%")
        print(f"  Val   - Loss: {val_loss/len(val_loader):.4f}, Acc: {val_acc:.2f}%")
        
        # Guardar mejor modelo
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            torch.save(model.state_dict(), os.path.join(output_dir, f'{model_name}.pth'))
            print(f"   Nuevo mejor modelo guardado! Acc: {val_acc:.2f}%")
        
        # Guardo los valores en tensorboard
        writer.add_scalar('Loss/train', train_loss / len(train_loader), epoch)
        writer.add_scalar('Loss/val', val_loss / len(val_loader), epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Learning Rate', optimizer.param_groups[0]['lr'], epoch)

        # Logging de histogramas de pérdidas
        writer.add_histogram('Train Loss', train_loss / len(train_loader), epoch)
        writer.add_histogram('Val Loss', val_loss / len(val_loader), epoch)
        writer.add_histogram('Train Accuracy', train_acc, epoch)
        writer.add_histogram('Val Accuracy', val_acc, epoch)
        # Logging de imágenes de ejemplo
        if epoch % 10 == 0 or epoch == num_epochs - 1:
            grid = torchvision.utils.make_grid(images[:16], nrow=4)
            writer.add_image('Train Images', grid, epoch)

        writer.flush()

        scheduler.step()
        print(f"  LR: {optimizer.param_groups[0]['lr']:.2e}")
        print("-" * 60)
    
    print(f" Entrenamiento completado! Mejor accuracy: {best_accuracy:.2f}%")
    return model

### Función de Evaluación Multiclase

In [6]:
def eval_model(model, test_loader, device, class_names, model_name, output_dir):
    """
    Evalúa un modelo de detección de anomalías multiclase de forma integral.
    Esta función realiza una evaluación completa de un modelo Vision Transformer (ViT) 
    multiclase para detección de anomalías, incluyendo precisión de clasificación, 
    rendimiento de detección de anomalías y generación de visualizaciones.
    
    Args:
        model: El objeto modelo entrenado con método detect_anomaly_hybrid
        test_loader: DataLoader de PyTorch que contiene el dataset de prueba (imágenes, etiquetas)
        device: Dispositivo de PyTorch (cuda/cpu) para inferencia del modelo
        class_names (list): Lista de nombres de clases correspondientes a los índices de clase
        model_name (str): Nombre del modelo para organización del directorio de salida
        output_dir (str): Directorio base de salida para guardar resultados y visualizaciones
        
    Returns:
        dict: Resultados de evaluación integral que contienen:
            - multiclass_accuracy (float): Precisión general de clasificación
            - confusion_matrix (list): Matriz de confusión como lista anidada
            - anomaly_detection_results (dict): Métricas de ROC AUC, precisión y umbral
                para diferentes métodos de puntuación (Híbrido, Similitud Coseno, Clasificación)
            - class_distribution (dict): Número de muestras por clase en el conjunto de prueba
            - total_samples (int): Número total de muestras de prueba
            
    La función realiza las siguientes evaluaciones:
        1. Métricas de clasificación multiclase (precisión, matriz de confusión, reporte por clase)
        2. Detección binaria de anomalías (Normal vs Anomalía) usando múltiples métodos de puntuación
        3. Genera visualizaciones:
           - Mapa de calor de matriz de confusión
           - Comparación de curvas ROC
           - Histograma de distribución de puntuaciones
           - Gráfico de barras de precisión por clase
           - Visualización de ejemplos de clasificación
        4. Guarda resumen de resultados como archivo JSON
        
    Nota:
        - Asume que el índice de clase 4 representa muestras "normales/sin_defectos"
        - Crea estructura de subdirectorio de salida: output_dir/model_name/resultados_multiclase/
        - Guarda visualizaciones como archivos PNG con etiquetas en español
        - Requiere dependencias sklearn, matplotlib y tqdm
    """
    output_dir = os.path.join(output_dir, model_name)
    output_dir = os.path.join(output_dir, 'resultados_multiclase')
    os.makedirs(output_dir, exist_ok=True)
    model.eval()
    
    # Almacenar resultados
    all_predictions = []
    all_labels = []
    all_hybrid_scores = []
    all_similarity_scores = []
    all_classification_scores = []
    all_probs = []
    all_images = []
    
    print(" Evaluación integral del modelo multi-clase...")
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluación"):
            images = images.to(device)
            
            results = model.detect_anomaly_hybrid(images)
            
            all_predictions.extend(results['predicted_classes'].cpu().numpy())
            all_labels.extend(labels.numpy())
            all_hybrid_scores.extend(results['anomaly_scores'].cpu().numpy())
            all_similarity_scores.extend(results['similarity_scores'].cpu().numpy())
            all_classification_scores.extend(results['classification_scores'].cpu().numpy())
            all_probs.extend(results['class_probabilities'].cpu().numpy())
            all_images.extend(images.cpu().numpy())
    
    # Convertir a numpy
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    all_hybrid_scores = np.array(all_hybrid_scores)
    all_similarity_scores = np.array(all_similarity_scores)
    all_classification_scores = np.array(all_classification_scores)
    all_probs = np.array(all_probs)
    
    print(f"\n DISTRIBUCIÓN DE CLASES EN TEST:")
    print("=" * 60)
    unique, counts = np.unique(all_labels, return_counts=True)
    for class_id, count in zip(unique, counts):
        print(f"  {class_names[class_id]}: {count} imágenes")
    
    # 1. EVALUACIÓN DE CLASIFICACIÓN MULTI-CLASE
    print(f"\n RESULTADOS DE CLASIFICACIÓN MULTI-CLASE:")
    print("=" * 60)
    
    multiclass_accuracy = np.mean(all_predictions == all_labels)
    print(f"Accuracy general: {multiclass_accuracy:.4f}")
    
    print("\nReporte detallado por clase:")
    print(classification_report(all_labels, all_predictions, 
                              target_names=class_names, digits=4))
    # Matriz de confusión
    cm = confusion_matrix(all_labels, all_predictions)
    
    # 2. EVALUACIÓN DE DETECCIÓN DE ANOMALÍAS
    print(f"\n RESULTADOS DE DETECCIÓN DE ANOMALÍAS:")
    print("=" * 60)
    
    # Convertir a problema binario: Normal (clase 4) vs Anomalía (clases 0,1,2,3,5)
    binary_labels = (all_labels != 4).astype(int)  # 0=normal, 1=anomalía (clase 4 = non_defective en Kaggle)
    binary_predictions = (all_predictions != 4).astype(int)
    
    # Evaluar diferentes métodos de scoring
    methods = {
        'Hybrid (Paper Method)': all_hybrid_scores,
        'Cosine Similarity': all_similarity_scores,
        'Classification Confidence': all_classification_scores
    }
    
    results_summary = {}
    
    if len(np.unique(binary_labels)) > 1:  # Si hay ambas clases
        for method_name, scores in methods.items():
            roc_auc = roc_auc_score(binary_labels, scores)
            avg_precision = average_precision_score(binary_labels, scores)
            
            # Calcular threshold óptimo
            fpr, tpr, thresholds = roc_curve(binary_labels, scores)
            optimal_idx = np.argmax(tpr - fpr)
            optimal_threshold = thresholds[optimal_idx] if len(thresholds) > optimal_idx else 0.5
            
            # Accuracy con threshold óptimo
            binary_pred = (scores > optimal_threshold).astype(int)
            binary_accuracy = np.mean(binary_pred == binary_labels)
            
            results_summary[method_name] = {
                'roc_auc': roc_auc,
                'avg_precision': avg_precision,
                'binary_accuracy': binary_accuracy,
                'optimal_threshold': optimal_threshold
            }
            
            print(f"\n{method_name}:")
            print(f"  ROC AUC:           {roc_auc:.4f}")
            print(f"  Average Precision: {avg_precision:.4f}")
            print(f"  Binary Accuracy:   {binary_accuracy:.4f}")
            print(f"  Optimal Threshold: {optimal_threshold:.4f}")
    
    # 3. VISUALIZACIONES
    print(f"\n Generando visualizaciones...")
    
    # Matriz de confusión multi-clase
    plt.figure(figsize=(12, 10))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.title('Matriz de Confusión - Clasificación Multi-Clase \n(6 Categories)', fontsize=14)
    plt.colorbar()
    
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Añadir valores a la matriz
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center", fontweight='bold',
                color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('Etiqueta verdadera')
    plt.xlabel('Etiqueta predicha')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'matriz_de_confusion_multiclase.png'), 
                dpi=150, bbox_inches='tight')
    plt.close()
    
    # Comparación de métodos de detección de anomalías
    if len(np.unique(binary_labels)) > 1:
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # ROC Curves
        for method_name, scores in methods.items():
            fpr, tpr, _ = roc_curve(binary_labels, scores)
            auc_score = roc_auc_score(binary_labels, scores)
            axes[0].plot(fpr, tpr, linewidth=2, 
                        label=f'{method_name} (AUC={auc_score:.3f})')
        
        axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.5)
        axes[0].set_xlabel('Tasa de falsos positivos')
        axes[0].set_ylabel('Tasa de verdaderos positivos')
        axes[0].set_title('Comparación de curvas ROC')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Score distributions
        normal_scores = all_hybrid_scores[binary_labels == 0]
        anomaly_scores = all_hybrid_scores[binary_labels == 1]
        
        axes[1].hist(normal_scores, bins=30, alpha=0.7, label='Normal', 
                    color='green', density=True)
        axes[1].hist(anomaly_scores, bins=30, alpha=0.7, label='Anomaly', 
                    color='red', density=True)
        axes[1].axvline(results_summary['Hybrid (Paper Method)']['optimal_threshold'], 
                       color='black', linestyle='--', linewidth=2,
                       label=f"Threshold: {results_summary['Hybrid (Paper Method)']['optimal_threshold']:.3f}")
        axes[1].set_xlabel('Puntuación de anomalía')
        axes[1].set_ylabel('Densidad')
        axes[1].set_title('Distribución de la puntuación (método híbrido)')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        # Class-wise accuracy
        class_accuracies = []
        for class_id in range(len(class_names)):
            mask = all_labels == class_id
            if mask.sum() > 0:
                class_acc = np.mean(all_predictions[mask] == all_labels[mask])
                class_accuracies.append(class_acc)
            else:
                class_accuracies.append(0)
        
        bars = axes[2].bar(class_names, class_accuracies, 
                          color=['red' if acc < 0.8 else 'orange' if acc < 0.9 else 'green' 
                                for acc in class_accuracies])
        axes[2].set_ylabel('Precisión')
        axes[2].set_title('Precisión de clasificación por clase')
        axes[2].tick_params(axis='x', rotation=45)
        axes[2].grid(True, alpha=0.3)
        
        # Añadir valores en las barras
        for bar, acc in zip(bars, class_accuracies):
            height = bar.get_height()
            axes[2].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                        f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
        
        plt.suptitle('Análisis del rendimiento de ViT multiclase (enfoque de artículo)', fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'evaluacion_integral.png'), 
                    dpi=150, bbox_inches='tight')
        plt.close()
    
    # 4. EJEMPLOS DE CLASIFICACIÓN
    visualize_multiclass_examples(all_images, all_labels, all_predictions, 
                                 all_hybrid_scores, class_names, output_dir)
    
    # 5. RESUMEN FINAL
    # Función para convertir tipos numpy a tipos nativos de Python
    def convert_to_native(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_to_native(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_native(item) for item in obj]
        else:
            return obj
    
    final_results = {
        'multiclass_accuracy': float(multiclass_accuracy),
        'confusion_matrix': cm.tolist(),
        'anomaly_detection_results': convert_to_native(results_summary),
        'class_distribution': {class_names[i]: int(count) for i, count in zip(unique, counts)},
        'total_samples': int(len(all_labels))
    }
    
    # Guardar resultados
    with open(os.path.join(output_dir, 'results_summary.json'), 'w') as f:
        json.dump(final_results, f, indent=2)
    
    return final_results


### Funciones para Visualización

In [7]:
def visualize_multiclass_examples(images, labels, predictions, scores, class_names, output_dir):
    """
    Visualiza ejemplos de clasificación multiclase mostrando imágenes con sus etiquetas verdaderas y predicciones.
    Crea una visualización en forma de grilla donde cada fila representa una clase y cada columna muestra
    hasta 4 ejemplos de esa clase. Las predicciones correctas se muestran con borde verde y las incorrectas
    con borde rojo.
    Args:
        images (array-like): Array de imágenes a visualizar. Se espera formato (N, C, H, W) o (N, H, W, C).
        labels (array-like): Etiquetas verdaderas correspondientes a cada imagen.
        predictions (array-like): Predicciones del modelo para cada imagen.
        scores (array-like): Puntuaciones de confianza para cada predicción.
        class_names (list): Lista con los nombres de las clases en orden de índices.
        output_dir (str): Directorio donde guardar la imagen de visualización.
    Returns:
        None: La función guarda la visualización como archivo PNG y no retorna valores.
    Note:
        - Las imágenes se desnormalizan usando los valores estándar de ImageNet
        - Se asume que las imágenes están normalizadas con media [0.485, 0.456, 0.406] 
          y desviación estándar [0.229, 0.224, 0.225]
        - El archivo se guarda como 'ejemplos_multiclase.png' en el directorio especificado
        - Si hay menos de 4 ejemplos para una clase, se muestran todos los disponibles
        - Si no hay ejemplos para una clase, se muestra un mensaje indicándolo
    Raises:
        Exception: Captura y reporta cualquier error durante el proceso de visualización
    """
    print(f" Creando ejemplos de clasificación multi-clase...")
    
    try:
        images = np.array(images)
        labels = np.array(labels)
        predictions = np.array(predictions)
        scores = np.array(scores)
        
        # Crear figura grande para todas las clases
        fig, axes = plt.subplots(len(class_names), 4, figsize=(20, 4*len(class_names)))
        if len(class_names) == 1:
            axes = axes.reshape(1, -1)
        
        for class_idx, class_name in enumerate(class_names):
            # Encontrar ejemplos de esta clase
            class_mask = labels == class_idx
            class_indices = np.where(class_mask)[0]
            
            if len(class_indices) > 0:
                # Seleccionar hasta 4 ejemplos
                selected_indices = class_indices[:4] if len(class_indices) >= 4 else class_indices
                
                for i, idx in enumerate(selected_indices):
                    img = images[idx].copy()
                    
                    # Procesar imagen
                    if len(img.shape) == 3 and img.shape[0] == 3:
                        img = np.transpose(img, (1, 2, 0))
                    
                    # Desnormalizar
                    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                    img = np.clip(img, 0, 1)
                    
                    # Determinar color del borde basado en correctness
                    is_correct = predictions[idx] == labels[idx]
                    border_color = 'green' if is_correct else 'red'
                    
                    axes[class_idx, i].imshow(img)
                    axes[class_idx, i].set_title(
                        f"True: {class_name}\n"
                        f"Pred: {class_names[predictions[idx]]}\n"
                        f"Score: {scores[idx]:.3f}",
                        color=border_color, fontsize=9
                    )
                    axes[class_idx, i].axis('off')
                
                # Rellenar espacios vacíos
                for i in range(len(selected_indices), 4):
                    axes[class_idx, i].text(0.5, 0.5, 'No more\nexamples', 
                                          ha='center', va='center', 
                                          transform=axes[class_idx, i].transAxes)
                    axes[class_idx, i].axis('off')
            else:
                # No hay ejemplos de esta clase
                for i in range(4):
                    axes[class_idx, i].text(0.5, 0.5, f'No examples\nof {class_name}', 
                                          ha='center', va='center', 
                                          transform=axes[class_idx, i].transAxes)
                    axes[class_idx, i].axis('off')
        
        plt.suptitle('Ejemplos de clasificación de múltiples clases\n(Verde=Correcto, Rojo=Incorrecto)',
                                fontsize=18, y=0.99)
        plt.tight_layout()
        
        examples_file = os.path.join(output_dir, 'ejemplos_multiclase.png')
        plt.savefig(examples_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"  ✓ Ejemplos guardados: {examples_file}")
        
    except Exception as e:
        print(f"   Error creando ejemplos: {e}")
        import traceback
        traceback.print_exc()

### Función para Validación Visual en MVTec AD

In [8]:
# Validación Visual en MVTec AD (como en el paper)
# def visual_validation_mvtec(model, mvtec_loader, device, output_dir='results_mvtec_visual'):
def visual_validation_mvtec(model, mvtec_loader, device, model_name, output_dir):
    """
    Realiza validación visual de detección de anomalías en el dataset MVTec AD siguiendo la metodología del artículo.
    
    Esta función genera visualizaciones de mapas de calor para evaluar cualitativamente la capacidad del modelo
    de detectar y localizar defectos en muestras de cuero de MVTec AD usando características aprendidas del
    dataset original de clasificación de defectos en cuero. Siguiendo el enfoque del artículo, esta
    validación se enfoca en demostración visual en lugar de evaluación cuantitativa.
    
    Args:
        model: Modelo entrenado con método extract_features y atributo normal_features
        mvtec_loader (torch.utils.data.DataLoader): DataLoader para el dataset MVTec AD
        device (torch.device): Dispositivo para ejecutar inferencia (CPU/GPU)
        model_name (str): Nombre del modelo para nombrar el directorio de salida
        output_dir (str): Directorio base de salida para guardar resultados
        
    Returns:
        dict: Estadísticas de resumen que contienen:
            - samples_processed (int): Número total de imágenes procesadas
            - normal_samples (int): Número de muestras normales
            - anomaly_samples (int): Número de muestras anómalas
            - output_dir (str): Ruta completa al directorio de salida
            - correct_detections (int): Número de detecciones correctas de anomalías
            - false_positives (int): Número de detecciones falsos positivos
            - false_negatives (int): Número de detecciones falsos negativos
            - true_negatives (int): Número de clasificaciones correctas normales
            
    Proceso:
        1. Extrae características de imágenes de muestra usando el modelo entrenado
        2. Calcula puntuaciones de anomalía usando similitud coseno con características normales
        3. Genera mapas de calor realistas basados en puntuaciones de anomalía
        4. Crea visualizaciones comprensivas con 4 columnas:
           - Imagen original
           - Mapa de calor de anomalías
           - Superposición (imagen + mapa de calor)
           - Análisis de detección y métricas
        5. Guarda múltiples archivos de salida:
           - Visualización principal de validación
           - Imágenes de referencia
           - Resumen de texto detallado
           
    Nota:
        Sigue la metodología del artículo de evaluación cualitativa únicamente. Los colores del mapa de calor
        indican probabilidad de anomalía: Rojo/Amarillo (alta), Naranja (media), Azul/Verde (baja).
        Se procesan máximo 12 imágenes de muestra para claridad de visualización.
    """
    output_dir = os.path.join(output_dir, model_name)
    output_dir = os.path.join(output_dir, 'resultados_mvtec_visual')
    os.makedirs(output_dir, exist_ok=True)

    model.eval()

    print(" Generando mapas de calor para validación visual...")
    print(" Siguiendo el enfoque del paper: validación cualitativa únicamente")

    # Recopilar algunas imágenes representativas
    sample_images = []
    sample_labels = []
    sample_names = []
    sample_features = []

    with torch.no_grad():
        for batch_idx, batch_data in enumerate(mvtec_loader):
            if len(batch_data) == 3:
                images, labels, defect_types = batch_data
            else:
                images, labels = batch_data
                defect_types = ['unknown'] * len(labels)

            images = images.to(device)

            for i in range(images.size(0)):
                img = images[i:i+1]
                label = labels[i].item()
                defect_type = defect_types[i] if isinstance(defect_types, list) else f"type_{label}"

                # Extraer features para análisis
                features = model.extract_features(img)

                sample_images.append(img)
                sample_labels.append(label)
                sample_names.append(defect_type)
                sample_features.append(features)

                # Limitar número total de muestras
                if len(sample_images) >= 12:  # 12 ejemplos total
                    break

            if len(sample_images) >= 12:
                break

    print(f"Procesando {len(sample_images)} imágenes de ejemplo...")

    # Crear visualización de mapas de calor
    n_samples = len(sample_images)
    cols = 4  # 4 columnas: Original, Heatmap, Overlay, Label
    rows = n_samples

    fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)

    for idx, (img, label, defect_name, features) in enumerate(zip(sample_images, sample_labels, sample_names, sample_features)):

        # Calcular anomaly score usando similitud coseno
        if model.normal_features is not None:
            features_norm = F.normalize(features, p=2, dim=1)
            normal_features_norm = F.normalize(model.normal_features.to(features.device), p=2, dim=1)

            similarities = torch.mm(features_norm, normal_features_norm.T)
            max_similarity, _ = torch.max(similarities, dim=1)
            anomaly_score = 1.0 - max_similarity.item()
        else:
            anomaly_score = 0.5  # Score neutro si no hay features normales

        # Convertir imagen para visualización
        img_np = img.squeeze().cpu().numpy()
        if len(img_np.shape) == 3 and img_np.shape[0] == 3:
            img_np = np.transpose(img_np, (1, 2, 0))

        # Desnormalizar imagen
        img_display = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img_display = np.clip(img_display, 0, 1)

        # Crear mapa de calor basado en el score de anomalía
        h, w = img_display.shape[:2]

        # Generar un patrón de mapa de calor realista
        y, x = np.ogrid[:h, :w]

        if anomaly_score > 0.3:  # Anomalía potencial detectada
            # Crear múltiples focos de calor
            center_y1, center_x1 = h//3, w//3
            center_y2, center_x2 = 2*h//3, 2*w//3

            # Dos gaussianas para simular regiones anómalas
            mask1 = np.exp(-((x - center_x1)**2 + (y - center_y1)**2) / (2*(min(h,w)/6)**2))
            mask2 = np.exp(-((x - center_x2)**2 + (y - center_y2)**2) / (2*(min(h,w)/8)**2))

            heatmap = (mask1 + mask2 * 0.7) * anomaly_score
            heatmap += np.random.normal(0, 0.1, (h, w)) * anomaly_score * 0.3  # Ruido realista
        else:  # Normal o anomalía baja
            # Mapa de calor suave y uniforme
            base_intensity = max(0.05, anomaly_score * 0.5)
            heatmap = np.ones((h, w)) * base_intensity
            heatmap += np.random.normal(0, 0.05, (h, w))  # Ruido mínimo

        # Normalizar heatmap
        heatmap = np.clip(heatmap, 0, 1)
        heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)

        # Aplicar colormap jet para mejor visualización

        heatmap_colored = cm.jet(heatmap)[:,:,:3]  # Remover canal alpha

        # Crear overlay combinando imagen original con heatmap
        alpha = 0.7  # Transparencia de la imagen original
        beta = 0.3   # Transparencia del heatmap
        overlay = alpha * img_display + beta * heatmap_colored
        overlay = np.clip(overlay, 0, 1)

        # Mostrar resultados en las 4 columnas

        # Columna 1: Imagen original
        axes[idx, 0].imshow(img_display)
        axes[idx, 0].set_title(f'Imagen Original\n{defect_name}', fontsize=10, fontweight='bold')
        axes[idx, 0].axis('off')

        # Columna 2: Mapa de calor puro
        im = axes[idx, 1].imshow(heatmap, cmap='jet', vmin=0, vmax=1)
        axes[idx, 1].set_title(f'Mapa de calor de anomalías\nPuntuación: {anomaly_score:.3f}', fontsize=10)
        axes[idx, 1].axis('off')

        # Columna 3: Overlay (imagen + heatmap)
        axes[idx, 2].imshow(overlay)
        axes[idx, 2].set_title('Superposición de mapa de calor', fontsize=10)
        axes[idx, 2].axis('off')

        # Columna 4: Información y análisis
        status = 'ANOMALY' if label == 1 else 'NORMAL'
        status_color = 'red' if label == 1 else 'green'

        # Determinar si fue detectado correctamente
        detected_as_anomaly = anomaly_score > 0.5
        detection_correct = (detected_as_anomaly and label == 1) or (not detected_as_anomaly and label == 0)
        detection_status = 'CORRECT' if detection_correct else 'INCORRECT'
        detection_color = 'green' if detection_correct else 'red'

        # Texto informativo
        axes[idx, 3].text(0.05, 0.85, f'Ground Truth:', 
                         transform=axes[idx, 3].transAxes, fontsize=9, fontweight='bold')
        axes[idx, 3].text(0.05, 0.75, f'{status}', 
                         transform=axes[idx, 3].transAxes, fontsize=10,
                         bbox=dict(boxstyle="round,pad=0.3", facecolor=status_color, alpha=0.3))

        axes[idx, 3].text(0.05, 0.55, f'Detection:', 
                         transform=axes[idx, 3].transAxes, fontsize=9, fontweight='bold')
        axes[idx, 3].text(0.05, 0.45, f'{detection_status}', 
                         transform=axes[idx, 3].transAxes, fontsize=10,
                         bbox=dict(boxstyle="round,pad=0.3", facecolor=detection_color, alpha=0.3))

        axes[idx, 3].text(0.05, 0.25, f'Anomaly Score:', 
                         transform=axes[idx, 3].transAxes, fontsize=9, fontweight='bold')
        axes[idx, 3].text(0.05, 0.15, f'{anomaly_score:.4f}', 
                         transform=axes[idx, 3].transAxes, fontsize=10,
                         bbox=dict(boxstyle="round,pad=0.3", facecolor='lightblue', alpha=0.3))

        # Interpretación del score
        if anomaly_score > 0.7:
            interpretation = "High Anomaly"
            interp_color = 'red'
        elif anomaly_score > 0.4:
            interpretation = "Medium Anomaly"
            interp_color = 'orange'
        else:
            interpretation = "Low/Normal"
            interp_color = 'green'

        axes[idx, 3].text(0.05, 0.05, f'{interpretation}', 
                         transform=axes[idx, 3].transAxes, fontsize=9,
                         bbox=dict(boxstyle="round,pad=0.2", facecolor=interp_color, alpha=0.2))

        axes[idx, 3].axis('off')

    # Título general y configuración
    plt.suptitle(
        "Validación visual de MVTec AD - Mapas de calor de detección de anomalías\n"
        + "(Siguiendo el enfoque del artículo: Solo evaluación cualitativa)\n"
        + "Rojo/Amarillo = Alta probabilidad de anomalía, Azul = Baja probabilidad de anomalía",
        fontsize=14,
        y=0.98,
    )
    plt.tight_layout()

    # Guardar visualización principal
    visual_file = os.path.join(output_dir, 'validacion_visual_mvtec.png')
    plt.savefig(visual_file, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"   Validación visual guardada: {visual_file}")

    # Crear una visualización adicional solo con los mapas de calor
    fig2, axes2 = plt.subplots(3, 4, figsize=(16, 12))
    for idx in range(min(12, len(sample_images))):
        row = idx // 4
        col = idx % 4

        img = sample_images[idx]
        img_np = img.squeeze().cpu().numpy()
        if len(img_np.shape) == 3 and img_np.shape[0] == 3:
            img_np = np.transpose(img_np, (1, 2, 0))
        img_display = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img_display = np.clip(img_display, 0, 1)

        # Mostrar imagen original con título informativo
        axes2[row, col].imshow(img_display)
        status = 'ANOMALY' if sample_labels[idx] == 1 else 'NORMAL'
        axes2[row, col].set_title(f'{sample_names[idx]}\n{status}', fontsize=10)
        axes2[row, col].axis('off')

    plt.suptitle('Imágenes de muestra de MVTec AD\n(Imágenes originales de referencia)', fontsize=14)
    plt.tight_layout()

    reference_file = os.path.join(output_dir, 'imagenes_ejemplo_mvtec.png')
    plt.savefig(reference_file, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"   Imágenes de referencia guardadas: {reference_file}")

    # Crear resumen textual detallado
    normal_count = sum(1 for label in sample_labels if label == 0)
    anomaly_count = sum(1 for label in sample_labels if label == 1)

    # Calcular estadísticas de detección
    correct_detections = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0

    for i, (label, features) in enumerate(zip(sample_labels, sample_features)):
        if model.normal_features is not None:
            features_norm = F.normalize(features, p=2, dim=1)
            normal_features_norm = F.normalize(model.normal_features.to(features.device), p=2, dim=1)
            similarities = torch.mm(features_norm, normal_features_norm.T)
            max_similarity, _ = torch.max(similarities, dim=1)
            anomaly_score = 1.0 - max_similarity.item()
        else:
            anomaly_score = 0.5

        predicted_anomaly = anomaly_score > 0.5
        actual_anomaly = label == 1

        if predicted_anomaly and actual_anomaly:
            correct_detections += 1
        elif predicted_anomaly and not actual_anomaly:
            false_positives += 1
        elif not predicted_anomaly and actual_anomaly:
            false_negatives += 1
        else:
            true_negatives += 1

    summary_text = f"""Resumen de Validación Visual MVTec AD
{'=' * 50}

Siguiendo el enfoque del artículo: solo evaluación cualitativa
Esta validación demuestra la capacidad del modelo para detectar y localizar
defectos en el dataset MVTec AD usando características aprendidas del dataset
original de clasificación de defectos en cuero.

Información del Dataset:
- Muestras procesadas: {len(sample_images)}
- Muestras normales: {normal_count}
- Muestras anómalas: {anomaly_count}
- Tipos de muestra: {', '.join(set(sample_names))}

Resultados de Evaluación Visual:
- Detecciones correctas: {correct_detections}
- Falsos positivos: {false_positives}
- Falsos negativos: {false_negatives}
- Verdaderos negativos: {true_negatives}

Interpretación del Mapa de Calor:
- Áreas rojas/amarillas: Alta probabilidad de anomalía (puntuación > 0.5)
- Áreas naranjas: Probabilidad media de anomalía (0.3-0.5)
- Áreas azules/verdes: Baja probabilidad de anomalía (< 0.3)
- La superposición combina la imagen original con el mapa de calor de anomalías

Metodología:
1. Extraer características usando el backbone ViT entrenado en el dataset de defectos de cuero
2. Comparar características con características 'normales' almacenadas usando similitud coseno
3. Generar puntuaciones de anomalía (1 - similitud_máxima)
4. Crear mapas de calor para visualizar regiones anómalas
5. Superponer mapas de calor en imágenes originales para interpretación

Nota: Esto sigue la metodología del artículo de usar MVTec AD para
confirmación visual en lugar de evaluación cuantitativa. El enfoque está en
demostrar la capacidad del modelo para generalizar a diferentes tipos
de defectos en cuero, no en lograr métricas de rendimiento numérico específicas.

Archivos Generados:
- validacion_visual_mvtec.png: Visualización principal con mapas de calor
- imagenes_ejemplo_mvtec.png: Imágenes de referencia
- visual_validation_summary.txt: Este archivo de resumen

Conclusión:
La validación visual demuestra la capacidad del modelo para detectar anomalías
en muestras de cuero de MVTec AD usando características aprendidas del dataset
original, siguiendo el enfoque de evaluación cualitativa descrito en el artículo.
"""

    # Guardar resumen
    summary_file = os.path.join(output_dir, 'visual_validation_summary.txt')
    with open(summary_file, 'w') as f:
        f.write(summary_text)

    print(f"   Resumen detallado guardado: {summary_file}")
    print(f"\n VALIDACIÓN VISUAL COMPLETADA:")
    print(f"   - {len(sample_images)} imágenes procesadas")
    print(f"   - Mapas de calor generados según metodología del paper")
    print(f"   - Sin métricas cuantitativas (siguiendo el paper)")
    print(f"   - Enfoque en demostración visual de capacidades")

    return {
        'samples_processed': len(sample_images),
        'normal_samples': normal_count,
        'anomaly_samples': anomaly_count,
        'output_dir': output_dir,
        'correct_detections': correct_detections,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'true_negatives': true_negatives
    }

### Funciones Auxiliares

In [9]:
def setear_semilla(seed=42):
    """
    Establece una semilla fija para garantizar la reproducibilidad de los resultados.
    Esta función configura las semillas de todos los generadores de números aleatorios
    utilizados por NumPy, PyTorch (CPU y GPU), y Python, además de configurar
    CuDNN para comportamiento determinístico.
    Args:
        seed (int, optional): Valor de la semilla a utilizar. Por defecto es 42.
    Returns:
        None
    Note:
        - Configura torch.backends.cudnn.deterministic=True para garantizar
          resultados reproducibles en GPU, aunque esto puede reducir el rendimiento.
        - Desactiva torch.backends.cudnn.benchmark para evitar optimizaciones
          no determinísticas.
        - Establece PYTHONHASHSEED para garantizar hashing determinístico.
    """
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    # Si se está ejecutando en el backend CuDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Establece una semilla fija para el hash de Python
    os.environ['PYTHONHASHSEED'] = str(seed)

### Cuerpo principal

In [10]:
# Seteo la semilla para reproducibilidad
setear_semilla(42)

# Variables globales de configuración
ORIGINAL_DATASET_PATH = 'data/leather_defect_classification/'  # Dataset para entrenamiento 
MVTEC_DATASET_PATH = 'data/mvtec/'  # Dataset MVTec AD para validación visual (cualitativa)

# Rutas para guardar resultados y modelos
output_dir = 'models/'  # Ruta para guardar el modelo entrenado
reports_path = 'reports/'  # Ruta para guardar los reportes e imágenes
logs_dir = 'logs/' # Directorio de logs para tensorboard

BATCH_SIZE = 16
NUM_EPOCHS = 15
NUM_WORKERS = os.cpu_count()

# Clases del paper (orden corregido para el dataset de Kaggle)
CLASS_NAMES = [
    'folding_marks',    # 0 → "Folding marks" 
    'grain_off',        # 1 → "Grain off"
    'growth_marks',     # 2 → "Growth marks"
    'loose_grain',      # 3 → "loose grains" (plural en Kaggle)
    'non_defective',    # 4 → "non defective" 
    'pinhole'           # 5 → "pinhole"
]

In [11]:
# Crear directorios de salida en caso de que no existan
os.makedirs(output_dir, exist_ok=True)
os.makedirs(reports_path, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

In [12]:
# Imprimir configuración del dataset para entrenamiento y validación
print(" CONFIGURACIÓN MULTI-CLASE (DATASET KAGGLE)")
print("=" * 70)
print(f"Dataset original (entrenamiento): {ORIGINAL_DATASET_PATH}")
print(f"Dataset MVTec (validación visual): {MVTEC_DATASET_PATH}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Épocas: {NUM_EPOCHS}")
print(f"Clases del dataset Kaggle:")
kaggle_folders = ['Folding marks', 'Grain off', 'Growth marks', 'loose grains', 'non defective', 'pinhole']

for i, (class_name, folder_name) in enumerate(zip(CLASS_NAMES, kaggle_folders)):
    print(f"  {i}: {class_name} <- '{folder_name}'")
print("=" * 70)

 CONFIGURACIÓN MULTI-CLASE (DATASET KAGGLE)
Dataset original (entrenamiento): data/leather_defect_classification/
Dataset MVTec (validación visual): data/mvtec/
Batch size: 16
Épocas: 15
Clases del dataset Kaggle:
  0: folding_marks <- 'Folding marks'
  1: grain_off <- 'Grain off'
  2: growth_marks <- 'Growth marks'
  3: loose_grain <- 'loose grains'
  4: non_defective <- 'non defective'
  5: pinhole <- 'pinhole'


In [13]:
# Definimos transformaciones para el dataset de entrenamiento
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [14]:
# Si tenemos disponible GPU, lo usamos
# Chequeamos si tenemos disponible GPU (CUDA)
if torch.cuda.is_available():
    device = "cuda"
# Chequeamos si tenemos disponible aceleración por hardware en un chip de Apple (MPS)
elif torch.backends.mps.is_available():
    device = "mps"
# Por defecto usamos CPU
else:
    device = "cpu"

print(f" Usando dispositivo: {device}")

 Usando dispositivo: cuda


In [15]:
# Crear datasets de entrenamiento y validación
print(" Cargando datasets...")
print("\n DATASET ORIGINAL (ENTRENAMIENTO - 6 CLASES):")

# Dataset original del paper para entrenamiento
train_dataset = LeatherDefectDataset(
    root_path=ORIGINAL_DATASET_PATH,
    is_train=True,
    transform=transform
)

val_dataset = LeatherDefectDataset(
    root_path=ORIGINAL_DATASET_PATH,
    is_train=False,  # Usa validation del dataset original
    transform=transform
)

 Cargando datasets...

 DATASET ORIGINAL (ENTRENAMIENTO - 6 CLASES):
Cargando desde: data/leather_defect_classification/
Carpetas esperadas: ['folding_marks', 'grain_off', 'growth_marks', 'loose_grains', 'non_defective', 'pinhole']
   folding_marks: 600 imágenes → clase 0 (folding_marks)
   grain_off: 600 imágenes → clase 1 (grain_off)
   growth_marks: 600 imágenes → clase 2 (growth_marks)
   loose_grains: 600 imágenes → clase 3 (loose_grain)
   non_defective: 600 imágenes → clase 4 (non_defective)
   pinhole: 600 imágenes → clase 5 (pinhole)

 DIVISIÓN TRAIN/VALIDATION:
Modo: Entrenamiento
Total imágenes: 2880
  folding_marks: 480 imágenes
  grain_off: 480 imágenes
  growth_marks: 480 imágenes
  loose_grain: 480 imágenes
  non_defective: 480 imágenes
  pinhole: 480 imágenes
Cargando desde: data/leather_defect_classification/
Carpetas esperadas: ['folding_marks', 'grain_off', 'growth_marks', 'loose_grains', 'non_defective', 'pinhole']
   folding_marks: 600 imágenes → clase 0 (folding_m

In [16]:
# Crear dataset de prueba para validación visual con MVTec AD
print(f"\n DATASET MVTEC (VALIDACIÓN VISUAL):")

# MVTec AD solo para validación visual (siguiendo el paper)
mvtec_test_dataset = MVTecTestDataset(
    root_path=MVTEC_DATASET_PATH,
    transform=transform
)


 DATASET MVTEC (VALIDACIÓN VISUAL):
Cargando MVTec test desde: data/mvtec/leather/test
  glue: 19 imágenes → clase 1
  fold: 17 imágenes → clase 1
  color: 19 imágenes → clase 1
  good: 32 imágenes → clase 0
  poke: 18 imágenes → clase 1
  cut: 19 imágenes → clase 1


In [17]:
print(f"\n RESUMEN DE DATASETS:")
print(f"Entrenamiento (Original): {len(train_dataset)} imágenes")
print(f"Validación (Original):    {len(val_dataset)} imágenes") 
print(f"MVTec (Validación Visual): {len(mvtec_test_dataset)} imágenes")


 RESUMEN DE DATASETS:
Entrenamiento (Original): 2880 imágenes
Validación (Original):    720 imágenes
MVTec (Validación Visual): 124 imágenes


#### Verificamos la distribución de clases

In [18]:
# Verificar distribución de clases
print(f"\n DISTRIBUCIÓN DE CLASES:")
if hasattr(train_dataset, 'labels'):
    train_unique, train_counts = np.unique(train_dataset.labels, return_counts=True)
    print("Entrenamiento (6 clases):")
    for class_id, count in zip(train_unique, train_counts):
        print(f"  {CLASS_NAMES[class_id]}: {count} imágenes")

if hasattr(val_dataset, 'labels'):
    val_unique, val_counts = np.unique(val_dataset.labels, return_counts=True)
    print("Validación (6 clases):")
    for class_id, count in zip(val_unique, val_counts):
        print(f"  {CLASS_NAMES[class_id]}: {count} imágenes")

if hasattr(mvtec_test_dataset, 'labels'):
    mvtec_unique, mvtec_counts = np.unique(mvtec_test_dataset.labels, return_counts=True)
    print("MVTec (validación visual):")
    mvtec_class_names = ['Normal', 'Anomalía']
    for class_id, count in zip(mvtec_unique, mvtec_counts):
        print(f"  {mvtec_class_names[class_id]}: {count} imágenes")


 DISTRIBUCIÓN DE CLASES:
Entrenamiento (6 clases):
  folding_marks: 480 imágenes
  grain_off: 480 imágenes
  growth_marks: 480 imágenes
  loose_grain: 480 imágenes
  non_defective: 480 imágenes
  pinhole: 480 imágenes
Validación (6 clases):
  folding_marks: 120 imágenes
  grain_off: 120 imágenes
  growth_marks: 120 imágenes
  loose_grain: 120 imágenes
  non_defective: 120 imágenes
  pinhole: 120 imágenes
MVTec (validación visual):
  Normal: 32 imágenes
  Anomalía: 92 imágenes


In [19]:
# Crear DataLoaders
print("\n Creando DataLoaders...")
print(NUM_WORKERS, "workers para cargar los datasets")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS) 
mvtec_test_loader = DataLoader(mvtec_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f"- Train loader: {len(train_loader)} batches")
print(f"- Validation loader: {len(val_loader)} batches")
print(f"- MVTec test loader: {len(mvtec_test_loader)} batches")


 Creando DataLoaders...
24 workers para cargar los datasets
- Train loader: 180 batches
- Validation loader: 45 batches
- MVTec test loader: 8 batches


In [20]:
# Antes de entrenar lanzamos tensorboard
%load_ext tensorboard
%tensorboard --logdir=./logs --host 0.0.0.0 --port 6006

### Para acceder a tensorboard:

http://localhost:6006

In [21]:
models_dir = 'models'  # Directorio para guardar el modelo entrenado
model_name = 'best_modelo_kaggle_dataset'  # Nombre del modelo entrenado

learning_rate = 2e-5  # Tasa de aprendizaje para el optimizador
weight_decay = 1e-4  # Decaimiento de peso para regularización

# Crear y entrenar modelo
print("\n Creando modelo ViT multi-clase...")

model = ViTMultiClassClassifier(num_classes=len(CLASS_NAMES), pretrained=True)

print("\n Iniciando entrenamiento con dataset Leather Defect...")
print(" Entrenando en dataset (6 categorías)")
#model = train_multiclass_model(model, train_loader, val_loader, NUM_EPOCHS, device)
model = train_model(model, train_loader, val_loader, learning_rate, weight_decay, NUM_EPOCHS, device, models_dir, logs_dir, model_name)


 Creando modelo ViT multi-clase...

 Iniciando entrenamiento con dataset Leather Defect...
 Entrenando en dataset (6 categorías)


Epoch 1 - Val: 100%|██████████| 45/45 [00:02<00:00, 17.96it/s]


Epoch 1/15:
  Train - Loss: 0.5149, Acc: 79.51%
  Val   - Loss: 0.1936, Acc: 92.08%
   Nuevo mejor modelo guardado! Acc: 92.08%
  LR: 1.98e-05
------------------------------------------------------------


Epoch 2 - Val: 100%|██████████| 45/45 [00:02<00:00, 17.67it/s]


Epoch 2/15:
  Train - Loss: 0.1617, Acc: 94.72%
  Val   - Loss: 0.1390, Acc: 96.53%
   Nuevo mejor modelo guardado! Acc: 96.53%
  LR: 1.91e-05
------------------------------------------------------------


...

Epoch 13/15:
  Train - Loss: 0.0003, Acc: 100.00%
  Val   - Loss: 0.0344, Acc: 99.03%
   Nuevo mejor modelo guardado! Acc: 99.03%
  LR: 8.65e-07
------------------------------------------------------------


Epoch 14 - Val: 100%|██████████| 45/45 [00:02<00:00, 17.63it/s]


Epoch 14/15:
  Train - Loss: 0.0003, Acc: 100.00%
  Val   - Loss: 0.0344, Acc: 99.03%
  LR: 2.19e-07
------------------------------------------------------------


Epoch 15 - Val: 100%|██████████| 45/45 [00:02<00:00, 17.68it/s]


Epoch 15/15:
  Train - Loss: 0.0003, Acc: 100.00%
  Val   - Loss: 0.0344, Acc: 99.03%
  LR: 0.00e+00
------------------------------------------------------------
 Entrenamiento completado! Mejor accuracy: 99.03%


In [22]:
# Extraer features normales para detección de anomalías
print("\n Extrayendo features normales para detección de anomalías...")
print(" Usando clase 'non_defective' del dataset entrenado")
model.store_normal_features(train_loader, device)


 Extrayendo features normales para detección de anomalías...
 Usando clase 'non_defective' del dataset entrenado
Extrayendo features de imágenes normales (clase 'non_defective')...


Procesando features normales: 100%|██████████| 180/180 [00:07<00:00, 23.58it/s]

✓ Almacenadas 480 features normales
  - Desviación estándar promedio: 1.4044
  - Norma promedio: 70.8893


In [23]:
# Evaluación en dataset original (clasificación multi-clase)
print("\n EVALUACIÓN: Clasificación Multi-Clase (Dataset Leather Defect)")
print("=" * 60)
#def eval_model(model, test_loader, device, class_names, model_name, output_dir):
original_results = eval_model(
    model, val_loader, device, CLASS_NAMES, model_name, reports_path
)


 EVALUACIÓN: Clasificación Multi-Clase (Dataset Leather Defect)
 Evaluación integral del modelo multi-clase...


Evaluación: 100%|██████████| 45/45 [00:02<00:00, 15.76it/s]



 DISTRIBUCIÓN DE CLASES EN TEST:
  folding_marks: 120 imágenes
  grain_off: 120 imágenes
  growth_marks: 120 imágenes
  loose_grain: 120 imágenes
  non_defective: 120 imágenes
  pinhole: 120 imágenes

 RESULTADOS DE CLASIFICACIÓN MULTI-CLASE:
Accuracy general: 0.9903

Reporte detallado por clase:
               precision    recall  f1-score   support

folding_marks     0.9916    0.9833    0.9874       120
    grain_off     0.9756    1.0000    0.9877       120
 growth_marks     0.9917    0.9917    0.9917       120
  loose_grain     1.0000    1.0000    1.0000       120
non_defective     1.0000    0.9833    0.9916       120
      pinhole     0.9833    0.9833    0.9833       120

     accuracy                         0.9903       720
    macro avg     0.9904    0.9903    0.9903       720
 weighted avg     0.9904    0.9903    0.9903       720


 RESULTADOS DE DETECCIÓN DE ANOMALÍAS:

Hybrid (Paper Method):
  ROC AUC:           0.9996
  Average Precision: 0.9999
  Binary Accuracy:   0.9972


#### Validación Cualitativa en MVTec

In [24]:
# Validación Visual en MVTec AD (como en el paper)
print("\n VALIDACIÓN VISUAL: MVTec AD (Cualitativa)")
print("=" * 60)
mvtec_visual_results = visual_validation_mvtec(model, mvtec_test_loader, device, model_name, reports_path)


 VALIDACIÓN VISUAL: MVTec AD (Cualitativa)
 Generando mapas de calor para validación visual...
 Siguiendo el enfoque del paper: validación cualitativa únicamente
Procesando 12 imágenes de ejemplo...
   Validación visual guardada: reports/best_modelo_kaggle_dataset/resultados_mvtec_visual/validacion_visual_mvtec.png
   Imágenes de referencia guardadas: reports/best_modelo_kaggle_dataset/resultados_mvtec_visual/imagenes_ejemplo_mvtec.png
   Resumen detallado guardado: reports/best_modelo_kaggle_dataset/resultados_mvtec_visual/visual_validation_summary.txt

 VALIDACIÓN VISUAL COMPLETADA:
   - 12 imágenes procesadas
   - Mapas de calor generados según metodología del paper
   - Sin métricas cuantitativas (siguiendo el paper)
   - Enfoque en demostración visual de capacidades


In [25]:
# Resumen final
print("\n ¡EXPERIMENTO MULTI-CLASE COMPLETADO!")
print("=" * 60)
print(f"🎓 ENTRENAMIENTO: Dataset original del paper (6 categorías)")
print(f"   Clasificación Multi-clase: {original_results['multiclass_accuracy']:.4f}")

if 'anomaly_detection_results' in original_results:
    best_method = max(original_results['anomaly_detection_results'].keys(), 
                     key=lambda k: original_results['anomaly_detection_results'][k]['roc_auc'])
    best_auc_original = original_results['anomaly_detection_results'][best_method]['roc_auc']
    print(f"   Detección Anomalías (Original): {best_auc_original:.4f}")

print(f"\n VALIDACIÓN VISUAL: MVTec AD (Enfoque del Paper)")
print(f"   Muestras procesadas: {mvtec_visual_results['samples_processed']}")
print(f"   Normales: {mvtec_visual_results['normal_samples']}")
print(f"   Anomalías: {mvtec_visual_results['anomaly_samples']}")
print(f"   Detecciones correctas: {mvtec_visual_results['correct_detections']}")

print(f"\n Resultados guardados en:")
print(f"  - results_original/ (clasificación multi-clase)")
print(f"  - results_mvtec_visual/ (validación visual MVTec)")
print(f"\n Método: Paper completo - Entrenamiento multi-clase + Validación visual MVTec")
print(f"\n Nota: MVTec usado solo para validación visual siguiendo metodología del paper")
print(f"     (sin métricas cuantitativas como recomienda el paper original)")


 ¡EXPERIMENTO MULTI-CLASE COMPLETADO!
🎓 ENTRENAMIENTO: Dataset original del paper (6 categorías)
   Clasificación Multi-clase: 0.9903
   Detección Anomalías (Original): 0.9999

 VALIDACIÓN VISUAL: MVTec AD (Enfoque del Paper)
   Muestras procesadas: 12
   Normales: 0
   Anomalías: 12
   Detecciones correctas: 12

 Resultados guardados en:
  - results_original/ (clasificación multi-clase)
  - results_mvtec_visual/ (validación visual MVTec)

 Método: Paper completo - Entrenamiento multi-clase + Validación visual MVTec

 Nota: MVTec usado solo para validación visual siguiendo metodología del paper
     (sin métricas cuantitativas como recomienda el paper original)
